In [3]:
# installing the mlflow if not installed

!pip install mlflow --quiet

In [2]:
# importing

import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [14]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [11]:
# loading and training a simple model

X, y = datasets.load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
params = {"solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

**Make sure to run the `mlflow ui` in the terminal !!!!!!**

In [16]:
# define an mlflow experiment

mlflow.set_experiment("first run")

2024/01/21 14:10:38 INFO mlflow.tracking.fluent: Experiment with name 'first run' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/861645051746955503', creation_time=1705826438911, experiment_id='861645051746955503', last_update_time=1705826438911, lifecycle_stage='active', name='first run', tags={}>

In [19]:
# start mlflow run

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.set_tag("Training Info", "Basic LR model for iris data")
    signature = infer_signature(X_train, lr.predict(X_train))
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

Successfully registered model 'tracking-quickstart'.
2024/01/21 14:16:44 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1
Created version '1' of model 'tracking-quickstart'.


In [20]:
# loading the saved model

loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [21]:
# using the model

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names
result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1


Thanks for reading